## Library

In [ ]:
import numpy as np
import pandas as pd
import zipfile
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings

warnings.filterwarnings(action="ignore")

DATA_PATH = "/kaggle/input/coupon-purchase-prediction/"

## EDA
* coupon_detail_train/test.csv 쿠폰 구매 정보  
  ITEM_COUNT: 쿠폰 개수  
  I_DATE: 구매일자  
  SMALL_AREA_NAME: 지역 정보  
  PURCHASEID_hash: 구매 해시값(PK)  
  USER_ID_hash: 유저 해시값  
  COUPON_ID_hash: 쿠폰 해시값  
    
* coupon_list_train/test.csv 모든 쿠폰 정보  
  CAPSULE_TEXT: 장르  
  GENRE_NAME: 장르  
  PRICE_RATE: 원가 대비 할인율  
  CATALOG_PRICE: 원가  
  DISCOUNT_PRICE: 할인되는 금액  
  DISPFROM: 게시 시작날짜  
  DISPEND: 게시 종료날짜  
  DISPPERIOD: 게시기간  
  VALIDFROM: 사용가능 시작날짜  
  VALIDEND: 사용가능 종료날짜  
  VALIDPERIOD: 사용가능 기간  
  USABLE_DATE_MON: 월요일 사용가능 여부  
  USABLE_DATE_TUE: 화요일 사용가능 여부  
  USABLE_DATE_WED: 수요일 사용가능 여부  
  USABLE_DATE_THU: 목요일 사용가능 여부  
  USABLE_DATE_FRI: 금요일 사용가능 여부  
  USABLE_DATE_SAT: 토요일 사용가능 여부  
  USABLE_DATE_SUN: 일요일 사용가능 여부  
  USABLE_DATE_HOLIDAY: 공휴일 사용가능 여부  
  USABLE_DATE_BEFORE_HOLIDAY: 공휴일 외 사용가능 여부  
  large_area_name: 사용가능 지역  
  ken_name  
  small_area_name  
  COUPON_ID_hash: 쿠폰 해시값(PK)  
    
* user_list.csv 회원 정보  
  REG_DATE: 등록일자  
  SEX_ID: 성별  
  AGE: 나이  
  WITHDRAW_DATE: 탈퇴일자  
  PREF_NAME: 선호지역  
  USER_ID_hash: 유저 해시값(PK)  
    
* coupon_visit_train/test.csv 사용자의 웹사이트 방문, 구매 기록  
  PURCHASE_FLG: 구매여부  
  I_DATE: 방문일시  
  PAGE_SERIAL: 방문 페이지 번호  
  REFERRER_hash: 방문 참조값 해시값 (방문전 링크값인거 같은데 활용 어떻게 할지 잘 모르겠음)  
  VIEW_COUPON_ID_hash: 확인한 쿠폰값 해시값  
  USER_ID_hash: 유저 해시값  
  SESSION_ID_hash: 세션 해시값  
  PURCHASEID_hash: 구매 해시값  

### data overview

In [ ]:
with zipfile.ZipFile(DATA_PATH + "coupon_detail_train.csv.zip", 'r') as zip_ref:
    # 압축 안의 파일 목록 확인
    file_list = zip_ref.namelist()
    print("압축 안의 파일:", file_list)
    # 첫 번째 CSV 파일을 DataFrame으로 읽기
    with zip_ref.open(file_list[0]) as file:
        df_detail = pd.read_csv(file)

# 결과 확인
df_detail.info()

In [ ]:
df_detail.head(1).T

In [ ]:
with zipfile.ZipFile(DATA_PATH + "coupon_list_train.csv.zip", 'r') as zip_ref:
    # 압축 안의 파일 목록 확인
    file_list = zip_ref.namelist()
    print("압축 안의 파일:", file_list)
    # 첫 번째 CSV 파일을 DataFrame으로 읽기
    with zip_ref.open(file_list[0]) as file:
        df_list = pd.read_csv(file)

# 결과 확인
df_list.info()


In [ ]:
df_list.head(1).T

In [ ]:
with zipfile.ZipFile(DATA_PATH + "coupon_visit_train.csv.zip", 'r') as zip_ref:
    # 압축 안의 파일 목록 확인
    file_list = zip_ref.namelist()
    print("압축 안의 파일:", file_list)
    # 첫 번째 CSV 파일을 DataFrame으로 읽기
    with zip_ref.open(file_list[0]) as file:
        df_visit = pd.read_csv(file)

# 결과 확인
df_visit.info()

In [ ]:
df_visit.head(1).T

In [ ]:
with zipfile.ZipFile(DATA_PATH + "user_list.csv.zip", 'r') as zip_ref:
    # 압축 안의 파일 목록 확인
    file_list = zip_ref.namelist()
    print("압축 안의 파일:", file_list)
    # 첫 번째 CSV 파일을 DataFrame으로 읽기
    with zip_ref.open(file_list[0]) as file:
        df_user = pd.read_csv(file)

# 결과 확인
df_user.info()

In [ ]:
df_user.head(1).T

### 결측치 확인

In [ ]:
print(f"<구매정보>\n{df_detail.isnull().sum()}\n")
print(f"<전체쿠폰정보>\n{df_list.isnull().sum()}\n")
print(f"<유저방문정보>\n{df_visit.isnull().sum()}\n")
print(f"<고객정보>\n{df_user.isnull().sum()}\n")

In [ ]:
print(df_detail.shape)
print(df_list.shape)
print(df_visit.shape)
print(df_user.shape)

## Preprocessing

In [ ]:
df_detail['I_DATE'] = pd.to_datetime(df_detail['I_DATE'])
df_detail['I_MONTH'] = df_detail['I_DATE'].dt.month
df_detail['I_DATE'] = df_detail['I_DATE'].dt.date

In [ ]:
df_detail.info()

In [ ]:
df_list['PURCHASE_PRICE'] = df_list['CATALOG_PRICE'] - df_list['DISCOUNT_PRICE']

In [ ]:
df_list.head(3)

### Merging: df_detail(구매기록) + df_list(쿠폰정보)

In [ ]:
df_detail.head(3)

In [ ]:
df_list.head(3)

In [ ]:
# PK확인
df_list.shape[0], len(df_list['COUPON_ID_hash'].unique())

In [ ]:
# PK확인
df_detail.shape[0], len(df_detail['PURCHASEID_hash'].unique())

In [ ]:
df = pd.merge(
    df_detail[['ITEM_COUNT','I_DATE','I_MONTH','SMALL_AREA_NAME','USER_ID_hash','COUPON_ID_hash', 'PURCHASEID_hash']],
    df_list[['GENRE_NAME','PRICE_RATE','PURCHASE_PRICE','COUPON_ID_hash']],
    on='COUPON_ID_hash',
    how='left'
)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df_detail['ITEM_COUNT'].max()

In [ ]:
# PK확인
df_user.shape[0], len(df_user['USER_ID_hash'].unique())

In [ ]:
df = pd.merge(
    df,
    df_user[['SEX_ID','AGE','USER_ID_hash']],
    on='USER_ID_hash',
    how='left'
)

In [ ]:
df.head().T

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

## RFM
구간분할 방식  
1. Quantile (33%, 66%)	상·중·하 분위로 3등분	균형적인 그룹 분할  
2. Natural break (분포기반)	실제 데이터 변곡점 기반 (예: 1회, 2~4회, 5회 이상)	해석력 높음  
3. Business rule	비즈니스적으로 의미 있는 기준	해석 명확

In [ ]:
last = df['I_DATE'].max() + pd.DateOffset(days=1) # 마지막 거래일자 다음날을 기준일로 설정

rfm_df = df.groupby(['USER_ID_hash']).agg({
    'I_DATE': lambda x:(last-pd.to_datetime(x.max())).days,
    'PURCHASEID_hash': lambda x:x.nunique(),
    'PURCHASE_PRICE': sum
})
rfm_df.rename(columns={'거래날짜':'Recency', '거래':'Frequency', '지불금액':'Monetary'},inplace=True)

rfm_df.head(3)

In [ ]:
last = df['I_DATE'].max() + pd.DateOffset(days=1) # 마지막 거래일자 다음날을 기준일로 설정

rfm_df = df.groupby(['USER_ID_hash']).agg({
    'I_DATE': lambda x:(last-pd.to_datetime(x.max())).days,
    'PURCHASEID_hash': lambda x:x.nunique(),
    'PURCHASE_PRICE': sum
})
rfm_df.rename(columns={'I_DATE':'Recency', 'PURCHASEID_hash':'Frequency', 'PURCHASE_PRICE':'Monetary'},inplace=True)

rfm_df.head(3)

In [ ]:
rfm_df.shape

### RFM 검증: Scatter plot
* 데이터 많을때는 찍지 말것
* RFM 분석을 하기 전에 실제 데이터가 그 가정을 따르는지를 확인용 (분석 전 진단용 시각화)
* 고객 행동이 전형적인 RFM 패턴을 따름 (R낮을수록 F,M이 높음)
* 고객 행동이 예측 가능한 구조이므로 RFM segmentation의 설명력이 올라감

In [ ]:
# 건수 많을 때는 전체 데이터셋으로 scatterplot 그리는 것 지양

plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
sns.scatterplot(data=rfm_df, x='Recency', y='Frequency')
plt.title('Recency vs Frequency')
plt.xlabel('Recency')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
sns.scatterplot(data=rfm_df, x='Recency', y='Monetary')
plt.title('Recency vs Monetary')
plt.xlabel('Recency')
plt.ylabel('Monetary')

plt.subplot(1, 3, 3)
sns.scatterplot(data=rfm_df, x='Frequency', y='Monetary')
plt.title('Frequency vs Monetary')
plt.xlabel('Frequency')
plt.ylabel('Monetary')

plt.tight_layout()
plt.show()

## RFM 분석결과

### Recency
* 0\~25일: 대다수의 고객 집중
* 25\~100일: 완만한 감소
* 100일\~: 거의 변화없음

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(data=rfm_df, x='Recency', bins=30, kde=True)
plt.title('Recency Distribution')
plt.xlabel('Recency (days)')
plt.ylabel('Count')
plt.grid(True)
plt.show()

In [ ]:
sns.histplot(np.log1p(rfm_df['Recency']), bins=30)
plt.title("Log-scaled Recency Distribution")

### Frequency
* 0\~5일: 대다수의 고객 집중
* 5\~40일: 완만한 감소
* 40일\~: 거의 변화없음

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(data=rfm_df, x='Frequency', bins=20, kde=False)
plt.title('Frequency Distribution')
plt.xlabel('Frequency')
plt.ylabel('Count')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(4,8))
sns.boxplot(data=rfm_df, y='Frequency')
plt.ylabel('count')
plt.show()

### Monetary
* 0\~0.7 (0\~70,000): 대다수의 고객 집중
* 0.7\~3.3 (70,000\~330,000): 완만한 감소
* 3.3\~ (330,000\~): 거의 변화없음

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(data=rfm_df, x='Monetary', bins=20, kde=False)
plt.title('Monetary Distribution')
plt.xlabel('spend amount')
plt.ylabel('Count')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(4,8))
sns.boxplot(data=rfm_df, y='Monetary')
plt.ylabel('count')
plt.show()

## RFM 점수계산

### recency grading

In [ ]:
'''
def assign_R(recency):
    if recency <= 30:
        return 5
    elif recency <= 60:
        return 4
    elif recency <= 90:
        return 3
    elif recency <= 180:
        return 2
    elif recency <= 365:
        return 1
    else:
        return 0
'''
# 일반적인 비즈니스 분할방식 30,60,90,180,365

In [ ]:
def assign_R(recency):
    if recency <= 25:
        return 3
    elif recency <= 100:
        return 2
    else:
        return 1

# rfm_df['R'] = assign_R(rfm['Recency'])는 틀림. 함수에 Recency값 전체를 넣기 때문
rfm_df['R'] = rfm_df['Recency'].apply(assign_R)

In [ ]:
rfm_df.head()

In [ ]:
rfm_df.groupby('R').count()

In [ ]:
rfm_df.groupby('R').mean()

### frequency grading

In [ ]:
plt.figure(figsize=(4,8))
sns.boxplot(data=rfm_df, y='Frequency')
plt.ylabel('count')
plt.show()

In [ ]:
rfm_df[rfm_df['Frequency'] >= 20].shape

In [ ]:
rfm_df['Frequency'].min(), rfm_df['Frequency'].max(), rfm_df['Frequency'].mean().round(2), rfm_df['Frequency'].median()

In [ ]:
rfm_df['Frequency_adj'] = rfm_df['Frequency'].apply(lambda x: 21 if x>=21 else x)

In [ ]:
# quantile binning(값이 비슷하게)
q_binned = pd.qcut(rfm_df['Frequency_adj'], q=3)
q_binned.value_counts().sort_index()

In [ ]:
# quantile binning(값이 비슷하게)
u_binned = pd.cut(rfm_df['Frequency_adj'], bins=20)
u_binned.value_counts().sort_index()

In [ ]:
# rfm_df['F'] = pd.cut(rfm_df['Frequency_adj'], bins=5, labels=[1,2,3,4,5])

In [ ]:
def assign_F(frequency):
    if frequency <= 5:
        return 1
    elif frequency <= 40:
        return 2
    else:
        return 3

rfm_df['F'] = rfm_df['Frequency'].apply(assign_F)

In [ ]:
rfm_df['F'].value_counts()

### Monetary grading

In [ ]:
def assign_M(Monetary):
    if Monetary <= 70000:
        return 1
    elif Monetary <= 330000:
        return 2
    else:
        return 3

rfm_df['M'] = rfm_df['Monetary'].apply(assign_M)

In [ ]:
rfm_df['M'].value_counts()

### RFM score 생성

In [ ]:
rfm_df['RFM_Score'] = rfm_df['R'] + rfm_df['F'] + rfm_df['M']

rfm_df.sample(5)

In [ ]:
rfm_df.groupby(['RFM_Score'])[['Recency', 'Frequency', 'Monetary']].mean()

In [ ]:
rfm_df['RFM_Score'].value_counts().sort_index()


## RFM 기반 고객 분류
* 3\~4: 이탈고객
* 5\~7: 잠재활성가능고객
* 8\~9: 충성고객
---
다만 중간층고객을 R/F/M 각각의 세부수치로 할지 그냥 RFM score로 할지는 고민중  
RFM score로만 하기에는 score:5인 집단이 조금 애매함  
그렇다고 세부수치로 하기에는 정확한 근거가 없음. RFM score별 집단 평균으로 해봤는데 평균이라 그런지 분류가 제대로 안 됨

In [ ]:
# 예시코드
'''
def classify_customer(row):
    r, f, m = row['R'], row['F'], row['M']

     # VIP
    if r >= 5 and f >= 5 and m >= 5:
        return 'VIP'
    # VIP이탈고객: Recency 점수가 3 이하이며, 나머지 점수가 4 이상
    elif r <= 3 and (f >= 4 and m >= 4):
        return 'VIP이탈고객'
    elif f >= 4 and r >= 4 and m >= 4:
        return '충성고객'
    elif f >= 3 and r >= 3 and m >= 3:
        return '충성예정고객'
    elif r <= 2 and f >= 3:
        return '충성이탈예정고객'
    else:
        return '일반고객'  # 이외의 경우를 위한 기본값
'''
# rfm_df['고객등급'] = rfm_df.apply(classify_customer, axis=1)

In [ ]:
def classify_customer(df):
    score = df['RFM_Score']
    r, f, m = df['Recency'], df['Frequency'], df['Monetary']
    
     # VIP
    if score >= 8:
        return '충성고객'
    # elif r<=38 and f>=14 and m>=57400:
    #     return '중간고객'
    elif score >= 5:
        return '일반고객'
    else :
        return '저활성고객'

rfm_df['고객등급'] = rfm_df.apply(classify_customer, axis=1)

In [ ]:
rfm_df.head()

In [ ]:
rfm_df.groupby(['고객등급'])[['Recency', 'Frequency', 'Monetary']].mean()

In [ ]:
rfm_df['고객등급'].value_counts()

## VIP 세그먼트 특징
* 구매행동지표  
  총구매금액  
  건당평균구매금액: 소비타입(고가/저가선호)  
  평균할인율: 소비타입(할인민감층/할인둔감층)  
  구매카테고리개수: 카테고리 다양도  
  구매지역패턴: 지역 다양도(small_area_name, 가장 세부적이므로 선호지역의 다양성을 확인하기 좋음)  
  재구매주기  
  ~~사용가능주기선호: 장기/단기 선호~~ 결측치 많음  
* 인구통계  
  성별  
  연령대  
  가입일
---
구매행동지표 -> df_detail(coupon_detail_train/test.csv), df_list(coupon_list_train/test.csv)  
인구통계 -> df_user(user_list.csv)

In [ ]:
# VIP 먼저 특징분석
vip_df = rfm_df[rfm_df['고객등급']=='충성고객'][['Recency', 'Frequency', 'Monetary', '고객등급']].copy()

In [ ]:
vip_df.sample()

In [ ]:
vip_df['고객등급'].unique()

In [ ]:
# USER_ID_hash가 index이므로 column으로 잠시 변환
vip_df = vip_df.reset_index()

In [ ]:
vip_df.sample()

### Merging(df_detail, df_list): 구매기록

In [ ]:
vip_df.shape[0], len(vip_df['USER_ID_hash'].unique())

In [ ]:
vip_purchase = vip_df.merge(df_detail[['ITEM_COUNT', 'I_DATE', 'USER_ID_hash', 'COUPON_ID_hash', 'PURCHASEID_hash']], 
                      on='USER_ID_hash', how='left')

In [ ]:
vip_purchase.shape[0], len(vip_purchase['PURCHASEID_hash'].unique())

In [ ]:
vip_purchase = vip_purchase.merge(df_list[['GENRE_NAME', 'PRICE_RATE', 'PURCHASE_PRICE', 'VALIDPERIOD', 'small_area_name', 'COUPON_ID_hash']],
                                 on='COUPON_ID_hash', how='left')

In [ ]:
vip_purchase.shape[0], len(vip_purchase['PURCHASEID_hash'].unique())

In [ ]:
vip_purchase.sample(3).T

In [ ]:
vip_purchase['GENRE_NAME'].unique()

In [ ]:
genre_en = {
    '宅配': 'Delivery service',
    'グルメ': 'Food',
    'ギフトカード': 'Gift card',
    'その他のクーポン': 'Other coupon',
    'レッスン': 'Lesson',
    'ホテル・旅館': 'Hotel and Japanese hotel',
    'エステ': 'Spa',
    'レジャー': 'Leisure',
    'ヘアサロン': 'Hair salon',
    'ネイル・アイ': 'Nail and eye salon',
    'リラクゼーション': 'Relaxation',
    'ビューティー': 'Beauty',
    '健康・医療': 'Health and medical'
}

vip_purchase['GENRE_NAME'] = vip_purchase['GENRE_NAME'].map(genre_en)

In [ ]:
vip_purchase['GENRE_NAME'].unique()

In [ ]:
vip_purchase['small_area_name'].unique()

In [ ]:
df_list['large_area_name'].unique()

In [ ]:
df_list['ken_name'].unique()

In [ ]:
small_area_en = {
    '銀座・新橋・東京・上野': 'Ginza / Shimbashi / Tokyo / Ueno',
    '滋賀': 'Shiga',
    '兵庫': 'Hyogo',
    '静岡': 'Shizuoka',
    '沖縄': 'Okinawa',
    '千葉': 'Chiba',
    '新宿・高田馬場・中野・吉祥寺': 'Shinjuku / Takadanobaba / Nakano / Kichijoji',
    '恵比寿・目黒・品川': 'Ebisu / Meguro / Shinagawa',
    '渋谷・青山・自由が丘': 'Shibuya / Aoyama / Jiyugaoka',
    '横浜': 'Yokohama',
    '北海道': 'Hokkaido',
    '池袋・神楽坂・赤羽': 'Ikebukuro / Kagurazaka / Akabane',
    '埼玉': 'Saitama',
    '愛媛': 'Ehime',
    'キタ': 'Kita',
    '京都': 'Kyoto',
    '岡山': 'Okayama',
    '愛知': 'Aichi',
    '和歌山': 'Wakayama',
    'ミナミ他': 'Minami area',
    '福岡': 'Fukuoka',
    '香川': 'Kagawa',
    '川崎・湘南・箱根他': 'Kawasaki / Shonan / Hakone',
    '赤坂・六本木・麻布': 'Akasaka / Roppongi / Azabu',
    '奈良': 'Nara',
    '岐阜': 'Gifu',
    '宮城': 'Miyagi',
    '新潟': 'Niigata',
    '三重': 'Mie',
    '群馬': 'Gunma',
    '茨城': 'Ibaraki',
    '広島': 'Hiroshima',
    '佐賀': 'Saga',
    '立川・町田・八王子他': 'Tachikawa / Machida / Hachioji',
    '長野': 'Nagano',
    '石川': 'Ishikawa',
    '長崎': 'Nagasaki',
    '福井': 'Fukui',
    '山口': 'Yamaguchi',
    '秋田': 'Akita',
    '宮崎': 'Miyazaki',
    '栃木': 'Tochigi',
    '富山': 'Toyama',
    '徳島': 'Tokushima',
    '鳥取': 'Tottori',
    '青森': 'Aomori',
    '鹿児島': 'Kagoshima',
    '福島': 'Fukushima',
    '大分': 'Oita',
    '高知': 'Kochi',
    '島根': 'Shimane',
    '山形': 'Yamagata',
    '山梨': 'Yamanashi',
    '岩手': 'Iwate',
    '熊本': 'Kumamoto'
}
vip_purchase['small_area_name'] = vip_purchase['small_area_name'].map(small_area_en)

In [ ]:
vip_purchase['small_area_name'].unique()

In [ ]:
vip_purchase.sample(3).T

In [ ]:
vip_purchase['I_DATE'].dtype

In [ ]:
vip_purchase['I_DATE'] = pd.to_datetime(vip_purchase['I_DATE'])

In [ ]:
vip_purchase['I_DATE'].dtype

### VIP 구매행동지표 계산
* 구매행동지표 (8가지)  
  구매횟수  
  구매쿠폰개수  
  총구매금액  
  평균구매금액  
  평균할인율  
  카테고리다양도  
  지역다양도  
  재구매주기

In [ ]:
vip_total_amount = vip_purchase.groupby('USER_ID_hash')['PURCHASE_PRICE'].sum()
vip_avg_amount = vip_purchase.groupby('USER_ID_hash')['PURCHASE_PRICE'].mean()
vip_avg_discount = vip_purchase.groupby('USER_ID_hash')['PRICE_RATE'].mean()
vip_order_count = vip_purchase.groupby('USER_ID_hash')['PURCHASEID_hash'].nunique()
vip_item_count = vip_purchase.groupby('USER_ID_hash')['ITEM_COUNT'].sum()
vip_category_unique = vip_purchase.groupby('USER_ID_hash')['GENRE_NAME'].nunique()
vip_area_unique = vip_purchase.groupby('USER_ID_hash')['small_area_name'].nunique()
def avg_repurchase_cycle(date):
    date = date.sort_values() # 날짜 정렬
    '''
    date.diff() # 인접 날짜끼리 차이 계산..! 이런 간단한 함수가 있었다니
    dropna() # 근데 처음과 끝은 인접날짜가 없으므로 NaT 널값이 나옴. 그거 제거
    dt.days() # 계산한 날짜차이를 days로 변환
    '''
    if len(date) <= 1: # 구매 횟수 1회인 경우는 계산불가
        return None
    return (date.diff().dropna().dt.days.mean())

vip_purchase_cycle = vip_purchase.groupby('USER_ID_hash')['I_DATE'].apply(avg_repurchase_cycle)

In [ ]:
vip_metrics = pd.DataFrame({
    'USER_ID_hash': vip_total_amount.index,
    '구매횟수': vip_order_count.values,
    '구매쿠폰개수': vip_item_count.values,
    '총구매금액': vip_total_amount.values,
    '평균구매금액': vip_avg_amount.values,
    '평균할인율': vip_avg_discount.values,
    '카테고리다양도': vip_category_unique.values,
    '지역다양도': vip_area_unique.values,
    '재구매주기': vip_purchase_cycle.values,
    # '평균사용기간': g_valid_period.values
})

In [ ]:
vip_metrics.sample(2).T

In [ ]:
vip_metrics[vip_metrics['구매횟수']==0].count()

### Merging(df_user): 인구통계 + 구매행동지표(vip_metrics)

In [ ]:
vip_df.shape[0], len(vip_df['USER_ID_hash'].unique())

In [ ]:
vip_df = vip_df.merge(df_user[['SEX_ID', 'AGE', 'USER_ID_hash']], 
                     on='USER_ID_hash', how='left')

In [ ]:
vip_df.shape[0], len(vip_df['USER_ID_hash'].unique())

In [ ]:
vip_df.sample(3)

In [ ]:
vip_df = vip_df.merge(vip_metrics, on='USER_ID_hash', how='left')

In [ ]:
vip_df.shape[0], len(vip_df['USER_ID_hash'].unique())

In [ ]:
vip_df.sample(3).T

In [ ]:
vip_df['AGE_GROUP'] = (vip_df['AGE']//10)*10

In [ ]:
vip_df.sample(3).T

In [ ]:
vip_df["AGE_GROUP"].value_counts()

In [ ]:
vip_df.groupby('AGE_GROUP')[[
    '구매횟수','구매쿠폰개수','총구매금액','평균구매금액','평균할인율','카테고리다양도','재구매주기'
]].mean().round(2)

### VIP 분석 시각화
1. Rader Chart: minmax, standard 적용
2. Heat Map: 각 지표는 서로 다른 단위라 지표간 비교는 불가  
   “구매금액_log = 0.35 → 전체 VIP 중 하위 35% 수준” (O)  
   “구매쿠폰개수 = 0.56 → 전체 VIP 중 상위 56% 수준” (O)  
   "구매금액_log = 0.35, 구매쿠폰개수 = 0.56 → 구매금액이 적고 쿠폰을 많이 샀다” (X)  
3. Parallel Coordinates

In [ ]:
import matplotlib.font_manager as fm

font_path = '/kaggle/input/font-kr/MALGUN.TTF'
fm.fontManager.addfont(font_path)
fontprop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = fontprop.get_name()
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
print(fontprop.get_name())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 3))
plt.plot([1, 2, 3], [10, 20, 15])
plt.title('한글 제목 테스트')
plt.xlabel('가로축')
plt.ylabel('세로축')
plt.show()

In [ ]:
cols_vip = [
    "구매횟수",
    "구매쿠폰개수",
    "총구매금액",
    "평균구매금액",
    "평균할인율",
    "카테고리다양도",
    "지역다양도"
]

vip_rader = vip_df.copy()

def age_group_func(age):
    if age < 30: return "20대"
    elif age < 40: return "30대"
    elif age < 50: return "40대"
    elif age < 60: return "50대"
    else: return "60대+"
vip_rader["AGE_GROUP2"] = vip_rader["AGE_GROUP"].apply(age_group_func)

groups = ["ALL", "20대", "30대", "40대", "50대", "60대+"]


# Rader Chart
def plot_radar(ax, values, labels, title):
    N = len(labels)

    # 각도 계산
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    angles += angles[:1]

    # 마지막 점을 첫 점으로 반복해서 폴리곤 닫기
    values = np.concatenate((values, [values[0]]))

    # 그리기
    ax.plot(angles, values, linewidth=2)
    ax.fill(angles, values, alpha=0.25)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, fontsize=9)
    ax.set_title(title, fontsize=12)

# 다른 지표간 비교를 위해 Scaling
scaler = MinMaxScaler()
# 이상치 너무 큰 값이 있으면 눌리는 현상이 나옴
minmax_scaled_all = scaler.fit_transform(vip_rader[cols_vip])
minmax_scaled_df = pd.DataFrame(minmax_scaled_all, columns=cols_vip)
minmax_scaled_df["AGE_GROUP2"] = vip_rader["AGE_GROUP2"]

# 그래프 그리기
fig, axes = plt.subplots(2, 3, figsize=(14, 10), subplot_kw=dict(polar=True))
axes = axes.flatten()

for idx, group in enumerate(groups):
    ax = axes[idx]
    if group == "ALL":
        radar_values = minmax_scaled_df[cols_vip].mean().values
    else:
        radar_values = minmax_scaled_df[minmax_scaled_df["AGE_GROUP2"] == group][cols_vip].mean().values

    plot_radar(ax, radar_values, cols_vip, f"VIP – {group}")

plt.tight_layout()
plt.show()

In [ ]:
cols_vip_mod = [
    "구매횟수",
    "구매쿠폰개수_log",
    "총구매금액_log",
    "평균구매금액_log",
    "평균할인율",
    "카테고리다양도",
    "지역다양도"
]

vip_rader = vip_df.copy()

def age_group_func(age):
    if age < 30: return "20대"
    elif age < 40: return "30대"
    elif age < 50: return "40대"
    elif age < 60: return "50대"
    else: return "60대+"
vip_rader["AGE_GROUP2"] = vip_rader["AGE_GROUP"].apply(age_group_func)

groups = ["ALL", "20대", "30대", "40대", "50대", "60대+"]


# 다른 지표간 비교를 위해 Scaling
scaler = MinMaxScaler()
# 이상치 너무 큰 값이 있으면 눌리는 현상이 나옴
vip_rader["총구매금액_log"] = np.log1p(vip_rader["총구매금액"])
vip_rader["평균구매금액_log"] = np.log1p(vip_rader["평균구매금액"])
vip_rader["구매쿠폰개수_log"] = np.log1p(vip_rader["구매쿠폰개수"])

minmax_scaled_all = scaler.fit_transform(vip_rader[cols_vip_mod])
minmax_scaled_df = pd.DataFrame(minmax_scaled_all, columns=cols_vip_mod)
minmax_scaled_df["AGE_GROUP2"] = vip_rader["AGE_GROUP2"]

# 그래프 그리기
fig, axes = plt.subplots(2, 3, figsize=(14, 10), subplot_kw=dict(polar=True))
axes = axes.flatten()

for idx, group in enumerate(groups):
    ax = axes[idx]
    if group == "ALL":
        radar_values = minmax_scaled_df[cols_vip_mod].mean().values
    else:
        radar_values = minmax_scaled_df[minmax_scaled_df["AGE_GROUP2"] == group][cols_vip_mod].mean().values

    plot_radar(ax, radar_values, cols_vip_mod, f"VIP – {group}")

plt.tight_layout()
plt.show()

In [ ]:
# Standard Scaler (연령대간 비교 불가. 각 연령대 내에서 지표들이 평균대비 얼마나 크고 작은지만 가능)
scaler = StandardScaler()
standard_scaled_all = scaler.fit_transform(vip_rader[cols_vip])
standard_scaled_df = pd.DataFrame(standard_scaled_all, columns=cols_vip)
standard_scaled_df["AGE_GROUP2"] = vip_rader["AGE_GROUP2"]

# 그래프 그리기
fig, axes = plt.subplots(2, 3, figsize=(14, 10), subplot_kw=dict(polar=True))
axes = axes.flatten()

for idx, group in enumerate(groups):
    ax = axes[idx]
    if group == "ALL":
        radar_values = standard_scaled_df[cols_vip].mean().values
    else:
        radar_values = standard_scaled_df[standard_scaled_df["AGE_GROUP2"] == group][cols_vip].mean().values

    plot_radar(ax, radar_values, cols_vip, f"VIP – {group}")

plt.tight_layout()
plt.show()

In [ ]:
# 다른 지표간 비교를 위해 Percentile Scaling 적용
for col in cols_vip:
    vip_rader[col + "_pct"] = vip_rader[col].rank(pct=True)

# 새 Scaling 컬럼
cols_pct = [c + "_pct" for c in cols_vip]

fig, axes = plt.subplots(2, 3, figsize=(14, 10), subplot_kw=dict(polar=True))
axes = axes.flatten()


def plot_radar_fixed_range(ax, values, labels, title):
    N = len(labels)

    # 각도 계산
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    angles += angles[:1]

    # 마지막 점을 첫 점으로 반복해서 폴리곤 닫기
    values = np.concatenate((values, [values[0]]))

    # 그리기
    ax.plot(angles, values, linewidth=2)
    ax.fill(angles, values, alpha=0.25)

    # 라벨
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, fontsize=9)
    ax.set_title(title, fontsize=12)

    # 범위 고정
    ax.set_ylim(0, 0.75)                       # 최소 0, 최대 1로 고정
    ax.set_yticks([0, 0.2, 0.4, 0.6, 0.75]) # 동일 tick
    ax.set_yticklabels(["0", "", "0.4", "0.6", "0.75"], fontsize=8)

for idx, group in enumerate(groups):
    ax = axes[idx]

    if group == "ALL":
        radar_values = vip_rader[cols_pct].mean().values
    else:
        radar_values = vip_rader[vip_rader["AGE_GROUP2"] == group][cols_pct].mean().values

    plot_radar_fixed_range(ax, radar_values, cols_vip, f"VIP고객 – {group}")

plt.tight_layout()
plt.show()

In [ ]:
pivot = minmax_scaled_df.groupby("AGE_GROUP2")[cols_vip_mod].mean()

plt.figure(figsize=(10,6))
sns.heatmap(pivot, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("VIP 행동지표 Heatmap (연령대별)", fontsize=14)
plt.ylabel("연령대")
plt.xlabel("행동지표")
plt.show()

In [ ]:
pivot = vip_rader.groupby("AGE_GROUP2")[cols_pct].mean()
pivot.loc["ALL"] = pivot[cols_pct].mean()

plt.figure(figsize=(10,6))
sns.heatmap(pivot, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("VIP 행동지표 percentile Heatmap (연령대별)", fontsize=14)
plt.ylabel("연령대")
plt.xlabel("행동지표")
plt.show()

## 일반고객 세그먼트 특징
* 구매행동지표  
  총구매금액  
  건당평균구매금액: 소비타입(고가/저가선호)  
  평균할인율: 소비타입(할인민감층/할인둔감층)  
  구매카테고리개수: 카테고리 다양도  
  구매지역패턴: 지역 다양도(small_area_name, 가장 세부적이므로 선호지역의 다양성을 확인하기 좋음)  
  재구매주기  
  ~~사용가능주기선호: 장기/단기 선호~~ 결측치 많음  
* 인구통계  
  성별  
  연령대  
  가입일
---
구매행동지표 -> df_detail(coupon_detail_train/test.csv), df_list(coupon_list_train/test.csv)  
인구통계 -> df_user(user_list.csv)

In [ ]:
medium_df = rfm_df[rfm_df['고객등급']=='일반고객'][['Recency', 'Frequency', 'Monetary', '고객등급']].copy()

In [ ]:
medium_df = medium_df.reset_index()

### Merging(df_detail, df_list): 구매기록

In [ ]:
medium_df.shape[0], len(medium_df['USER_ID_hash'].unique())

In [ ]:
medium_purchase = medium_df.merge(df_detail[['ITEM_COUNT', 'I_DATE', 'USER_ID_hash', 'COUPON_ID_hash', 'PURCHASEID_hash']], 
                      on='USER_ID_hash', how='left')

In [ ]:
medium_purchase.shape[0], len(medium_purchase['PURCHASEID_hash'].unique())

In [ ]:
medium_purchase = medium_purchase.merge(df_list[['GENRE_NAME', 'PRICE_RATE', 'PURCHASE_PRICE', 'VALIDPERIOD', 'small_area_name', 'COUPON_ID_hash']],
                                 on='COUPON_ID_hash', how='left')

In [ ]:
medium_purchase.shape[0], len(medium_purchase['PURCHASEID_hash'].unique())

In [ ]:
medium_purchase['GENRE_NAME'] = medium_purchase['GENRE_NAME'].map(genre_en)
medium_purchase['GENRE_NAME'].unique()

In [ ]:
medium_purchase['small_area_name'] = medium_purchase['small_area_name'].map(small_area_en)
medium_purchase['small_area_name'].unique()

In [ ]:
medium_purchase['I_DATE'].dtype

In [ ]:
medium_purchase['I_DATE'] = pd.to_datetime(medium_purchase['I_DATE'])

In [ ]:
medium_purchase['I_DATE'].dtype

### 일반고객 연령별 카테고리 구성비중
* 이 세그먼트 고객 중 몇 %가 이 카테고리를 1번이라도 사용했는가?  
  -> 고객들이 어떤 카테고리에 “관심”이 있는지  
* 거래량 & 금액 기준 카테고리 사용률: 실제로 어떤 카테고리에서 돈이 나오는지

In [ ]:
medium_purchase.shape[0], len(medium_purchase['PURCHASEID_hash'].unique())

In [ ]:
medium_purchase.sample(3).T

In [ ]:
# AGE_GROUP join (연령 정보 추가)
medium_purchase = medium_purchase.merge(
    df_user[['USER_ID_hash', 'AGE']],
    on='USER_ID_hash',
    how='left'
)
medium_purchase.shape[0], len(medium_purchase['PURCHASEID_hash'].unique())

In [ ]:
medium_purchase['AGE_GROUP'] = (medium_purchase['AGE']//10)*10
medium_purchase["AGE_GROUP"] = medium_purchase["AGE_GROUP"].apply(age_group_func)

In [ ]:
medium_purchase.sample(3).T

In [ ]:
# 사용자 기반(User-based) 연령대별 카테고리 관심도 (1번이상  구매)
medium_user_category = (
    medium_purchase.groupby(['AGE_GROUP', 'USER_ID_hash', 'GENRE_NAME'])['PURCHASEID_hash']
        .nunique()
        .reset_index()
)

medium_user_category['used'] = 1

# pivot → 연령대 × 카테고리 (사용자 비율)
medium_user_category_interest = (
    medium_user_category
        .pivot_table(index=['AGE_GROUP', 'USER_ID_hash'],
                     columns='GENRE_NAME',
                     values='used',
                     fill_value=0)
        .groupby(level=0)  # AGE_GROUP 단위로 묶기
        .mean()            # 사용자 평균 → 사용자 비율(User %)
        .sort_index()
)

In [ ]:
print("1번이상 구매한 카테고리들")
medium_user_category_interest.loc['60대+'].sort_values(ascending=False)

In [ ]:
medium_user_category_volume = (
    medium_purchase.groupby(['AGE_GROUP', 'GENRE_NAME'])['PURCHASEID_hash']
                .nunique()   # 구매 건수 기준
                .groupby(level=0)
                .apply(lambda x: x / x.sum())   # 연령대별 정규화
)

In [ ]:
print("구매량 기준 카테고리 정렬")
medium_user_category_volume.loc['60대+'].sort_values(ascending=False)

In [ ]:
medium_user_category_revenue = (
    medium_purchase.groupby(['AGE_GROUP', 'GENRE_NAME'])['PURCHASE_PRICE']
        .sum()
        .reset_index()
)

medium_revenue_share = (
    medium_user_category_revenue
        .groupby('AGE_GROUP')
        .apply(lambda x: x.set_index('GENRE_NAME')['PURCHASE_PRICE'] /
                        x['PURCHASE_PRICE'].sum())
        .unstack(level='AGE_GROUP')
        .fillna(0)
)

In [ ]:
print("사용금액 기준 카테고리 정렬")
medium_revenue_share.xs('60대+', level='AGE_GROUP').sort_values(ascending=False)

### 일반고객 구매행동지표 계산
* 구매행동지표 (8가지)  
  구매횟수  
  구매쿠폰개수  
  총구매금액  
  평균구매금액  
  평균할인율  
  카테고리다양도  
  지역다양도  
  재구매주기

In [ ]:
def get_purchase_metrics(df_purchase):
    df_total_amount = df_purchase.groupby('USER_ID_hash')['PURCHASE_PRICE'].sum()
    df_avg_amount = df_purchase.groupby('USER_ID_hash')['PURCHASE_PRICE'].mean()
    df_avg_discount = df_purchase.groupby('USER_ID_hash')['PRICE_RATE'].mean()
    df_order_count = df_purchase.groupby('USER_ID_hash')['PURCHASEID_hash'].nunique()
    df_item_count = df_purchase.groupby('USER_ID_hash')['ITEM_COUNT'].sum()
    df_category_unique = df_purchase.groupby('USER_ID_hash')['GENRE_NAME'].nunique()
    df_area_unique = df_purchase.groupby('USER_ID_hash')['small_area_name'].nunique()
    df_purchase_cycle = df_purchase.groupby('USER_ID_hash')['I_DATE'].apply(avg_repurchase_cycle)

    df_metrics = pd.DataFrame({
        'USER_ID_hash': df_total_amount.index,
        '구매횟수': df_order_count.values,
        '구매쿠폰개수': df_item_count.values,
        '총구매금액': df_total_amount.values,
        '평균구매금액': df_avg_amount.values,
        '평균할인율': df_avg_discount.values,
        '카테고리다양도': df_category_unique.values,
        '지역다양도': df_area_unique.values,
        '재구매주기': df_purchase_cycle.values,
        # '평균사용기간': g_valid_period.values
    })

    return df_metrics

In [ ]:
medium_metrics = get_purchase_metrics(medium_purchase)
medium_metrics.sample(3).T

In [ ]:
medium_metrics[medium_metrics['구매횟수']==0].count()

### Merging(df_user): 인구통계 + 구매행동지표(medium_metrics)

In [ ]:
medium_df.shape[0], len(medium_df['USER_ID_hash'].unique())

In [ ]:
medium_df = medium_df.merge(df_user[['SEX_ID', 'AGE', 'USER_ID_hash']], 
                     on='USER_ID_hash', how='left')

In [ ]:
medium_df.shape[0], len(medium_df['USER_ID_hash'].unique())

In [ ]:
medium_df.sample(3)

In [ ]:
medium_df = medium_df.merge(medium_metrics, on='USER_ID_hash', how='left')
medium_df.shape[0], len(medium_df['USER_ID_hash'].unique())

In [ ]:
medium_df.sample(3).T

In [ ]:
medium_df['AGE_GROUP'] = (medium_df['AGE']//10)*10
medium_df.sample(3).T

In [ ]:
medium_df["AGE_GROUP"].value_counts()

In [ ]:
medium_df.groupby('AGE_GROUP')[[
    '구매횟수','구매쿠폰개수','총구매금액','평균구매금액','평균할인율','카테고리다양도','재구매주기'
]].mean().round(2)

### 일반고객 분석 시각화
1. Rader Chart: percentile scaling 적용
2. Heat Map  
   같은 지표 내에서 연령대 간 상대적 위치를 비교  
   같은 연령대에서 지표 간 비교는 불가   
3. Parallel Coordinates

In [ ]:
cols_medium = [
    "구매횟수",
    "구매쿠폰개수",
    "총구매금액",
    "평균구매금액",
    "평균할인율",
    "카테고리다양도",
    "지역다양도"
] 

medium_rader = medium_df.copy()
medium_rader["AGE_GROUP2"] = medium_rader["AGE_GROUP"].apply(age_group_func)
groups = ["ALL", "20대", "30대", "40대", "50대", "60대+"]


# 다른 지표간 비교를 위해 Percentile Scaling 적용
for col in cols_medium:
    medium_rader[col + "_pct"] = medium_rader[col].rank(pct=True)

# 새 Scaling 컬럼
cols_pct = [c + "_pct" for c in cols_medium]

fig, axes = plt.subplots(2, 3, figsize=(14, 10), subplot_kw=dict(polar=True))
axes = axes.flatten()


def plot_radar_fixed_range(ax, values, labels, title):
    N = len(labels)

    # 각도 계산
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    angles += angles[:1]

    # 마지막 점을 첫 점으로 반복해서 폴리곤 닫기
    values = np.concatenate((values, [values[0]]))

    # 그리기
    ax.plot(angles, values, linewidth=2)
    ax.fill(angles, values, alpha=0.25)

    # 라벨
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, fontsize=9)
    ax.set_title(title, fontsize=12)

    # 범위 고정
    ax.set_ylim(0, 0.75)                       # 최소 0, 최대 1로 고정
    ax.set_yticks([0, 0.2, 0.4, 0.6, 0.75]) # 동일 tick
    ax.set_yticklabels(["0", "", "0.4", "0.6", "0.75"], fontsize=8)

for idx, group in enumerate(groups):
    ax = axes[idx]

    if group == "ALL":
        radar_values = medium_rader[cols_pct].mean().values
    else:
        radar_values = medium_rader[medium_rader["AGE_GROUP2"] == group][cols_pct].mean().values

    plot_radar_fixed_range(ax, radar_values, cols_medium, f"일반고객 – {group}")

plt.tight_layout()
plt.show()

In [ ]:
pivot = medium_rader.groupby("AGE_GROUP2")[cols_pct].mean()
pivot.loc["ALL"] = pivot[cols_pct].mean()

plt.figure(figsize=(10,6))
sns.heatmap(pivot, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("일반고객 행동지표 Heatmap (연령대별)", fontsize=14)
plt.ylabel("연령대")
plt.xlabel("행동지표")
plt.show()

## 비활성고객 세그먼트 특징

In [ ]:
low_df = rfm_df[rfm_df['고객등급']=='저활성고객'][['Recency', 'Frequency', 'Monetary', '고객등급']].copy()

In [ ]:
low_df = low_df.reset_index()

### Merging(df_detail, df_list): 구매기록

In [ ]:
low_df.shape[0], len(low_df['USER_ID_hash'].unique())

In [ ]:
low_purchase = low_df.merge(df_detail[['ITEM_COUNT', 'I_DATE', 'USER_ID_hash', 'COUPON_ID_hash', 'PURCHASEID_hash']], 
                      on='USER_ID_hash', how='left')
low_purchase.shape[0], len(low_purchase['PURCHASEID_hash'].unique())

In [ ]:
low_purchase = low_purchase.merge(df_list[['GENRE_NAME', 'PRICE_RATE', 'PURCHASE_PRICE', 'VALIDPERIOD', 'small_area_name', 'COUPON_ID_hash']],
                                 on='COUPON_ID_hash', how='left')
low_purchase.shape[0], len(low_purchase['PURCHASEID_hash'].unique())

In [ ]:
low_purchase['GENRE_NAME'] = low_purchase['GENRE_NAME'].map(genre_en)
low_purchase['GENRE_NAME'].unique()

In [ ]:
low_purchase['small_area_name'] = low_purchase['small_area_name'].map(small_area_en)
low_purchase['small_area_name'].unique()

In [ ]:
low_purchase['I_DATE'].dtype

In [ ]:
low_purchase['I_DATE'] = pd.to_datetime(low_purchase['I_DATE'])
low_purchase['I_DATE'].dtype

### 비활성고객 연령별 카테고리 구성비중
* 이 세그먼트 고객 중 몇 %가 이 카테고리를 1번이라도 사용했는가?  
  -> 고객들이 어떤 카테고리에 “관심”이 있는지  
* 거래량 & 금액 기준 카테고리 사용률: 실제로 어떤 카테고리에서 돈이 나오는지

In [ ]:
low_purchase.shape[0], len(low_purchase['PURCHASEID_hash'].unique())

In [ ]:
low_purchase.sample(3).T

In [ ]:
# AGE_GROUP join (연령 정보 추가)
low_purchase = low_purchase.merge(
    df_user[['USER_ID_hash', 'AGE']],
    on='USER_ID_hash',
    how='left'
)
low_purchase.shape[0], len(low_purchase['PURCHASEID_hash'].unique())

In [ ]:
low_purchase['AGE_GROUP'] = (low_purchase['AGE']//10)*10

In [ ]:
low_purchase.sample(3).T

In [ ]:
# 사용자 기반(User-based) 연령대별 카테고리 관심도 (1번이상  구매)
low_user_category = (
    low_purchase.groupby(['AGE_GROUP', 'USER_ID_hash', 'GENRE_NAME'])['PURCHASEID_hash']
        .nunique()
        .reset_index()
)

low_user_category['used'] = 1

# pivot → 연령대 × 카테고리 (사용자 비율)
low_user_category_interest = (
    low_user_category
        .pivot_table(index=['AGE_GROUP', 'USER_ID_hash'],
                     columns='GENRE_NAME',
                     values='used',
                     fill_value=0)
        .groupby(level=0)  # AGE_GROUP 단위로 묶기
        .mean()            # 사용자 평균 → 사용자 비율(User %)
        .sort_index()
)

In [ ]:
print("1번이상 구매한 카테고리들")
low_user_category_interest.loc[50].sort_values(ascending=False)

In [ ]:
low_user_category_volume = (
    low_purchase.groupby(['AGE_GROUP', 'GENRE_NAME'])['PURCHASEID_hash']
                .nunique()   # 구매 건수 기준
                .groupby(level=0)
                .apply(lambda x: x / x.sum())   # 연령대별 정규화
)

In [ ]:
print("구매량 기준 카테고리 정렬")
low_user_category_volume.loc[50].sort_values(ascending=False)

In [ ]:
low_user_category_revenue = (
    low_purchase.groupby(['AGE_GROUP', 'GENRE_NAME'])['PURCHASE_PRICE']
        .sum()
        .reset_index()
)

In [ ]:
low_revenue_share = (
    low_user_category_revenue
        .groupby('AGE_GROUP')
        .apply(lambda x: x.set_index('GENRE_NAME')['PURCHASE_PRICE'] /
                        x['PURCHASE_PRICE'].sum())
        .unstack()
        .fillna(0)
)

In [ ]:
print("사용금액 기준 카테고리 정렬")
low_revenue_share.loc[50].sort_values(ascending=False)

### 비활성고객 구매행동지표 계산
* 구매행동지표 (8가지)  
  구매횟수  
  구매쿠폰개수  
  총구매금액  
  평균구매금액  
  평균할인율  
  카테고리다양도  
  지역다양도  
  재구매주기

In [ ]:
low_metrics = get_purchase_metrics(low_purchase)
low_metrics.sample(3).T

In [ ]:
low_metrics[low_metrics['구매횟수']==0].count()

### Merging(df_user): 인구통계 + 구매행동지표(low_metrics)

In [ ]:
low_df.shape[0], len(low_df['USER_ID_hash'].unique())

In [ ]:
low_df = low_df.merge(df_user[['SEX_ID', 'AGE', 'USER_ID_hash']], 
                     on='USER_ID_hash', how='left')
low_df.shape[0], len(low_df['USER_ID_hash'].unique())

In [ ]:
low_df.sample(3)

In [ ]:
low_df = low_df.merge(low_metrics, on='USER_ID_hash', how='left')
low_df.shape[0], len(low_df['USER_ID_hash'].unique())

In [ ]:
low_df.sample(3).T

In [ ]:
low_df['AGE_GROUP'] = (low_df['AGE']//10)*10
low_df.sample(3).T

In [ ]:
low_df["AGE_GROUP"].value_counts()

In [ ]:
low_df.groupby('AGE_GROUP')[[
    '구매횟수','구매쿠폰개수','총구매금액','평균구매금액','평균할인율','카테고리다양도','재구매주기'
]].mean().round(2)

### 비활성고객 분석 시각화
1. Rader Chart
2. Heat Map

In [ ]:
cols_low = [
    "구매횟수",
    "구매쿠폰개수",
    "총구매금액",
    "평균구매금액",
    "평균할인율",
    "카테고리다양도",
    "지역다양도"
] 

low_rader = low_df.copy()
low_rader["AGE_GROUP2"] = low_rader["AGE_GROUP"].apply(age_group_func)
groups = ["ALL", "20대", "30대", "40대", "50대", "60대+"]


# 다른 지표간 비교를 위해 Percentile Scaling 적용
for col in cols_low:
    low_rader[col + "_pct"] = low_rader[col].rank(pct=True)

# 새 Scaling 컬럼
cols_pct = [c + "_pct" for c in cols_low]

fig, axes = plt.subplots(2, 3, figsize=(14, 10), subplot_kw=dict(polar=True))
axes = axes.flatten()


for idx, group in enumerate(groups):
    ax = axes[idx]

    if group == "ALL":
        radar_values = low_rader[cols_pct].mean().values
    else:
        radar_values = low_rader[low_rader["AGE_GROUP2"] == group][cols_pct].mean().values

    plot_radar_fixed_range(ax, radar_values, cols_low, f"비활성고객 – {group}")

plt.tight_layout()
plt.show()

In [ ]:
pivot = low_rader.groupby("AGE_GROUP2")[cols_pct].mean()
pivot.loc["ALL"] = pivot[cols_pct].mean()

plt.figure(figsize=(10,6))
sns.heatmap(pivot, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("비활성고객 행동지표 Heatmap (연령대별)", fontsize=14)
plt.ylabel("연령대")
plt.xlabel("행동지표")
plt.show()

## 세그먼트별 전체 비교

In [ ]:
vip_rader["고객등급"] = "VIP"
medium_rader["고객등급"] = "일반고객"
low_rader["고객등급"] = "저활성고객"

all_rader=  pd.concat([vip_rader, medium_rader, low_rader], axis=0, ignore_index=True)

In [ ]:
all_rader.sample(3).T

In [ ]:
cols_pct = [c+"_pct" for c in cols_low]
all_rader.drop(columns=cols_pct, axis=1, errors='ignore', inplace=True)
all_rader.sample(3).T

In [ ]:
cols_all = [
    "구매횟수",
    "구매쿠폰개수",
    "총구매금액",
    "평균구매금액",
    "평균할인율",
    "카테고리다양도",
    "지역다양도"
]

for col in cols_all:
    all_rader[col + "_pct"] = all_rader[col].rank(pct=True)

all_rader.sample(3).T

In [ ]:
all_pivot = all_rader.groupby("고객등급")[cols_pct].mean()
all_pivot

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(all_pivot, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("전체 세그먼트별 행동지표 비교 Heatmap", fontsize=14)
plt.xlabel("행동지표")
plt.ylabel("세그먼트")
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, polar=True)

# 값
angles = np.linspace(0, 2*np.pi, len(cols_all), endpoint=False).tolist()
angles += angles[:1]

# 범위 고정
ax.set_ylim(0, 1)
ax.set_yticks([0, 0.25, 0.5, 0.75, 1])

for seg, color in zip(["VIP", "일반고객", "저활성고객"], ["red", "blue", "green"]):
    values = all_pivot.loc[seg].values
    values = np.concatenate((values, [values[0]]))

    ax.plot(angles, values, linewidth=2, label=seg, color=color)
    ax.fill(angles, values, alpha=0.1, color=color)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(cols_all, fontsize=10)
plt.legend(loc="upper right")
plt.title("VIP / Medium / Low 행동 비교 Radar Chart", fontsize=14)
plt.show()


## Cohort 분석
* 가입월이 2011-07인 사람들의 시간대별 분석
* Cohort Analysis - Retention Rates  
  연속해석 X: 연속 생존율 (이번달 생존인원|저번달 생존인원) 아님  
  각달 개별 해석 O: 기준 그룹대비 각 달별로 다시 돌아온 비율 (100%/40%/30% = 100/40/12 아니고 100/40/30)

In [ ]:
df['I_DATE'].dtype

In [ ]:
df['I_DATE'] = pd.to_datetime(df['I_DATE'])
df['I_DATE'].dtype

In [ ]:
# 주문발생 연/월
df['order_month'] = df['I_DATE'].dt.to_period('M')

In [ ]:
user_cohort = (
    df.groupby('USER_ID_hash')['order_month']
      .min()
      .rename('cohort_month') # 기준월이 코호트
)

df = df.join(user_cohort, on='USER_ID_hash')

In [ ]:
# period 타입 → 숫자 차이(개월) 계산
def cohort_period_to_index(row):
    cohort = row['cohort_month']
    order  = row['order_month']
    return (order.year - cohort.year) * 12 + (order.month - cohort.month)

df['cohort_index'] = df.apply(cohort_period_to_index, axis=1)

In [ ]:
# 1) 코호트월 × 경과개월별 active user 수
cohort_data = (
    df
    .groupby(['cohort_month', 'cohort_index'])['USER_ID_hash']
    .nunique()
    .reset_index(name='active_users')
)

# 2) 각 코호트의 초기 유저 수 (0개월차 기준)
cohort_sizes = (
    cohort_data[cohort_data['cohort_index'] == 0]
    [['cohort_month', 'active_users']]
    .rename(columns={'active_users': 'cohort_size'})
)

# 3) Merge 리텐션 계산
cohort_data = cohort_data.merge(cohort_sizes, on='cohort_month')
cohort_data['retention'] = cohort_data['active_users'] / cohort_data['cohort_size']

# 4) 히트맵용 피벗 (행: 코호트월, 열: 경과개월)
retention_pivot = cohort_data.pivot(
    index='cohort_month',
    columns='cohort_index',
    values='retention'
)

# 보기 좋게 index를 문자열로 바꾸고 싶으면 (선택)
retention_pivot.index = retention_pivot.index.astype(str)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(18, 6))
sns.heatmap(
    retention_pivot,
    annot=True,          # 각 칸에 숫자 표시
    fmt='.0%',           # 0.12 → '12%'
    cmap='Blues',        # 색상
    vmin=0, vmax=1       # 0~1 스케일 고정
)

plt.title('Cohort Analysis - Retention Rates', fontsize=16)
plt.ylabel('Cohort Group')
plt.xlabel('Months After First Purchase')
plt.yticks(rotation=0)
plt.show()